# AI Agent using LangChain

In [127]:
# !pip install -q langchain-openai langchain-community langchain-core requests duckduckgo-search

## **Step 1:** Import Libraries

In [128]:
import os, requests
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

# Load environment variables
load_dotenv()

True

## **Step 2:** Setup Tools

### Tool 1: Search Tool

In [129]:
search_tool = DuckDuckGoSearchRun()
search_tool.invoke('Top news in Pakistan Today')

'Stay on top of Pakistan latest developments on the ground with Al Jazeera\'s fact-based news, exclusive video footage, photos and updated maps. India\'s aerial strikes on Pakistan began at 1:05 a.m. today (3:35 p.m. ET Tuesday) and lasted for 25 minutes, Col. Sofiya Qureshi, an Indian army officer said at a news briefing, adding no ... Pakistan says it has killed dozens of Indian soldiers in Kashmir, as it responds to Indian airstrikes. India targeted what it called "terrorist infrastructure" yesterday after an attack in its ... Punjab Today News Live Updates: India-Pakistan tensions hit tourism industry hard in Amritsar India\'s one of the most prominent tourist and spiritual destinations, Amritsar is witnessing a sharp decline in tourist footfall amid the Indo-Pak flare-up, rendering scores of people associated with the hospitality industry jobless and without income. Pakistan said the strikes hit at least two sites previously tied to banned militant groups. One hit the Subhan Mosqu

### Tool 2: Weather Tool

In [130]:
WEATHERSTACK_API_KEY = os.getenv("WEATHERSTACK_API_KEY")

@tool
def get_weather_data(city: str) -> str:
  """
  This function fetches the current weather data for a given city
  """
  url = f'https://api.weatherstack.com/current?access_key={WEATHERSTACK_API_KEY}&query={city}'

  response = requests.get(url)

  return response.json()

### Tool 3: Curreny Tool

In [131]:
from pydantic import BaseModel

EXCHANGE_RATE_API_KEY = os.getenv("EXCHANGE_RATE_API_KEY")

class CurrencyPair(BaseModel):
    pair: str  # e.g. "USD, PKR"

@tool(args_schema = CurrencyPair)
def get_currency(pair: str) -> float:
    """
    Get the exchange rate for a currency pair given as "BASE, TARGET".
    Example input: "USD, PKR"
    """
    base_currency, target_currency = [c.strip() for c in pair.split(",")]
    url = f"https://v6.exchangerate-api.com/v6/{EXCHANGE_RATE_API_KEY}/pair/{base_currency}/{target_currency}"
    response = requests.get(url)
    return response.json()

# @tool
# def get_currency(base_currency: str, target_currency: str) -> float:
#     """ This tool fetches the conversion factor between two currencies using an external API.
#     """
#     url = f"https://v6.exchangerate-api.com/v6/{EXCHANGE_RATE_API_KEY}/pair/{base_currency}/{target_currency}"
#     response = requests.get(url)
#     return response.json()["conversion_rate"]

## **Step 3:** Grab ReACT Pattern Prompt

In [132]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

# Step 2: Pull the ReAct prompt from LangChain Hub
prompt = hub.pull("hwchase17/react")  # pulls the standard ReAct agent prompt
# prompt = hub.pull("hwchase17/react-chat")


c:\Users\DELL\miniconda3\envs\llm_environment\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


## **Step 4:** Define LLM | Agent | Agent Executor

* **LLM**: Its used for reasoning
* **Agent**: It has `LLM + Tools`. It creates a plan on what to do
* **Agent Executor**: It executes the plan the `Agent` decides

In [133]:
# Create LLM Instance
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [134]:
# Step 3: Create the ReAct agent manually with the pulled prompt
agent = create_react_agent(
    llm=llm,
    tools=[search_tool, get_weather_data, get_currency],
    prompt=prompt
)

# Step 4: Wrap it with AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool, get_weather_data, get_currency],
    handle_parsing_errors=True,
    verbose=True
)

In [135]:
# Step 5: Invoke
response = agent_executor.invoke({"input": "What's the current weather in Faisalabad? Also what's the exchange rate of USD to PKR?"})
print(response)
print("--" * 50)
print(response['output'])



> Entering new AgentExecutor chain...
I need to gather the current weather data for Faisalabad and the exchange rate for USD to PKR. I'll start with the weather data.

Action: get_weather_data  
Action Input: "Faisalabad"  {'request': {'type': 'City', 'query': 'Faisalabad, Pakistan', 'language': 'en', 'unit': 'm'}, 'location': {'name': 'Faisalabad', 'country': 'Pakistan', 'region': 'Punjab', 'lat': '31.417', 'lon': '73.083', 'timezone_id': 'Asia/Karachi', 'localtime': '2025-05-10 03:22', 'localtime_epoch': 1746847320, 'utc_offset': '5.0'}, 'current': {'observation_time': '10:22 PM', 'temperature': 29, 'weather_code': 176, 'weather_icons': ['https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0025_light_rain_showers_night.png'], 'weather_descriptions': ['Patchy rain nearby'], 'astro': {'sunrise': '05:15 AM', 'sunset': '06:53 PM', 'moonrise': '04:58 PM', 'moonset': '03:41 AM', 'moon_phase': 'Waxing Gibbous', 'moon_illumination': 94}, 'air_quality': {'co': '2290.3', 'no2